# [Using Hooks or Git like actions](https://docs.lakefs.io/hooks/)

## This notebook demonstrated how to create a pre-merge hook in lakeFS that validates the schema files before merging them into the production branch. 

### 1. Define a hook configuration file and a Lua script for schema validation. 
### 2. Perform an ETL process by creating an ingestion branch, defining the table schema, and creating a table and atomically promoted the data to the production branch through a merge.
### 3. Attempt to change the schema of the table and promote it to production again. 
### 4. The pre-merge hook prevented the promotion due to schema changes, resulting in a Precondition Failed error.


![Actions UI](./Images/LuaHooks/schemaValidationFlow.png)

## Prerequisites

###### This Notebook requires connecting to a lakeFS Server. 
###### To spin up lakeFS quickly - spin up a playground environment on lakeFS Cloud (https://lakefs.cloud) which provides lakeFS server on-demand with a single click; 
###### Or, alternatively, refer to lakeFS Quickstart doc (https://docs.lakefs.io/quickstart/installing.html).

## Setup Task: Change your lakeFS credentials

In [ ]:
lakefsEndPoint = 'https://' # e.g. 'https://YOURLAKEFS.lakefscloud.io' or 'http://host.docker.internal:8000' (if lakeFS is running in local Docker container)
lakefsAccessKey = ''
lakefsSecretKey = ''

## Setup Task: You can change lakeFS repo name (it can be an existing repo or provide a new repo name)

In [ ]:
repo = "schema-validation-example-repo"
mainBranch = "main"
ingestionBranch = "ingestion_branch"

## Setup Task: Storage Information 
Note: If you spun up a lakeFS Cloud instance, and have not yet connected your storage, copy the storage namespace of the sample repository and append a string to it.

In [ ]:
storageNamespace = 's3://iddos3/blogpostschemavalidation/random/kjsdte3d7/' # e.g. "s3://username-lakefs-cloud/"

## Setup Task: Import Python packages

In [ ]:
import lakefs_client
from lakefs_client import models
from lakefs_client.client import LakeFSClient

import os
from pyspark.sql.types import ByteType, IntegerType, LongType, StringType, StructType, StructField

## Setup Task: Working with the lakeFS Python client API

###### Note: To learn more about lakeFS Python integration visit https://docs.lakefs.io/integrations/python.html

In [ ]:
%xmode Minimal
if not 'client' in locals():
    # lakeFS credentials and endpoint
    configuration = lakefs_client.Configuration()
    configuration.username = lakefsAccessKey
    configuration.password = lakefsSecretKey
    configuration.host = lakefsEndPoint

    client = LakeFSClient(configuration)
    print("Created lakeFS client.")

## Setup Task: Run PySpark with the Delta Lake package and additional configurations

In [ ]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:2.0.0 --conf "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension" --conf "spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog" pyspark-shell'

## Setup Task: S3A Gateway configuration

##### Note: lakeFS can be configured to work with Spark in two ways:
###### * Access lakeFS using the S3A gateway https://docs.lakefs.io/integrations/spark.html#use-the-s3-gateway.
###### * Access lakeFS using the lakeFS-specific Hadoop FileSystem https://docs.lakefs.io/integrations/spark.html#use-the-lakefs-hadoop-filesystem.

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", lakefsAccessKey)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", lakefsSecretKey)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", lakefsEndPoint)
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")

## Setup Task: Create lakeFS Repository

In [ ]:
client.repositories.create_repository(
    repository_creation=models.RepositoryCreation(
        name=repo,
        storage_namespace=storageNamespace,
        default_branch=mainBranch))

## Setup Task: Upload [Hooks config YAML file](./LuaHooks/pre-merge-schema-validation.yaml) for schema validation to check for any schema changes before data is merged to main branch

### Hooks config file must be uploaded to "_lakefs_actions" prefix

In [ ]:
hooks_config_yaml = "pre-merge-schema-validation.yaml"
hooks_prefix = "_lakefs_actions"
with open(f'./LuaHooks/{hooks_config_yaml}', 'rb') as f:
    client.objects.upload_object(repository=repo, 
                                 branch=mainBranch, 
                                 path=f'{hooks_prefix}/{hooks_config_yaml}', 
                                 content=f
                                )

## Setup Task: Upload [Schema Change script](./LuaHooks/parquet_schema_change.lua) to check for any schema changes

In [ ]:
lua_script_file_name = "parquet_schema_change.lua"
lua_scripts_path = "scripts"

In [ ]:
with open(f'./LuaHooks/{lua_script_file_name}', 'rb') as f:
    client.objects.upload_object(repository=repo, 
                                 branch=mainBranch, 
                                 path=f'{lua_scripts_path}/{lua_script_file_name}', 
                                 content=f
                                )

## Setup Task: Commit changes and attach some metadata

In [ ]:
client.commits.commit(
    repository=repo,
    branch=mainBranch,
    commit_creation=models.CommitCreation(
        message='Added hooks config file and schema validation script'))

# ETL Job Starts

## Create a new branch which will be used to ingest data

In [ ]:
client.branches.create_branch(
    repository=repo, 
    branch_creation=models.BranchCreation(
        name=ingestionBranch, source=mainBranch))

## For this demo - we'll be utilizing a dataset - [Orion Star - Sports and outdoors RDBMS dataset](https://www.kaggle.com/datasets/chethanp11/orion-star-sports-and-outdoors-rdbms-dataset) from [Kaggle](https://www.kaggle.com/).

## Define [CUSTOMER.csv](../data/samples/OrionStar/CUSTOMER.csv) data file schema

In [ ]:
customersSchema = StructType([
  StructField("User_ID", IntegerType(), False), 
  StructField("Country", StringType(), False),
  StructField("Gender", StringType(), False),
  StructField("Personal_ID", IntegerType(), True),
  StructField("Customer_Name", StringType(), False),
  StructField("Customer_FirstName", StringType(), False),
  StructField("Customer_LastName", StringType(), False),
  StructField("Birth_Date", StringType(), False),
  StructField("Customer_Address", StringType(), False),
  StructField("Street_ID", LongType(), False),
  StructField("Street_Number", IntegerType(), False),
  StructField("Customer_Type_ID", IntegerType(), False)
])

## Create Customers delta table in the new branch (using [CUSTOMER.csv](./data/samples/OrionStar/CUSTOMER.csv) file)

In [ ]:
customersTablePath = f"s3a://{repo}/{ingestionBranch}/tables/customers"
df = spark.read.csv('./data/samples/OrionStar/CUSTOMER.csv',header=True,schema=customersSchema)
df.write.format("delta").mode("overwrite").save(customersTablePath)
df.show(10)

## Commit changes and attach some metadata

In [ ]:
client.commits.commit(
    repository=repo,
    branch=ingestionBranch,
    commit_creation=models.CommitCreation(
        message='Added customers Delta table', 
        metadata={'using': 'python_api'}))

## Promote the Data into production

#### Merging the ingestion branch with the current schema to the production branch

In [ ]:
client.refs.merge_into_branch(
    repository=repo,
    source_ref=ingestionBranch, 
    destination_branch=mainBranch)

# On the next ETL Cycle - Change the schema and try to promote new data

## Change "Country" column to "Country_Name" in the schema

In [ ]:
customersSchema = StructType([
  StructField("User_ID", IntegerType(), False),
  StructField("Country_Name", StringType(), False), # Column name changes from Country to Country_name
  StructField("Gender", StringType(), False),
  StructField("Personal_ID", IntegerType(), True),
  StructField("Customer_Name", StringType(), False),
  StructField("Customer_FirstName", StringType(), False),
  StructField("Customer_LastName", StringType(), False),
  StructField("Birth_Date", StringType(), False),
  StructField("Customer_Address", StringType(), False),
  StructField("Street_ID", LongType(), False),
  StructField("Street_Number", IntegerType(), False),
  StructField("Customer_Type_ID", IntegerType(), False)
])

## Create Customers delta table in the new branch (using [CUSTOMER.csv](./data/samples/OrionStar/CUSTOMER.csv) file)

In [ ]:
customersTablePath = f"s3a://{repo}/{ingestionBranch}/tables/customers"
df = spark.read.csv('./data/samples/OrionStar/CUSTOMER.csv',header=True,schema=customersSchema)
df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save(customersTablePath)
df.show(10)

## Commit changes and attach some metadata

In [ ]:
client.commits.commit(
    repository=repo,
    branch=ingestionBranch,
    commit_creation=models.CommitCreation(
        message='Added customers Delta tables with schema changes!', 
        metadata={'using': 'python_api'}))

## Merge new branch to the main branch

#### Merge will fail because schema changed. Review the error message.

In [ ]:
client.refs.merge_into_branch(
    repository=repo,
    source_ref=ingestionBranch, 
    destination_branch=mainBranch)

## You can also review all Actions in lakeFS UI

![Actions UI](./Images/LuaHooks/SchemaValidation.gif)

# Want to learn about more advanced use cases with hooks?

## Try out the broader "Hooks Schema Validation.ipynb" notebook in https://github.com/treeverse/lakeFS-samples/tree/main/03-multiple-samples that incorporates also preventing promotion of PII data using hooks

## More Questions?

###### Join the lakeFS Slack group - https://lakefs.io/slack